# Fine-tuning Llama-2 Model with Deepspeed on Intel Gaudi

In this Jupyter notebook, we will fine-tune a [Llama-2-70b](https://huggingface.co/meta-llama/Llama-2-70b-chat-hf) model with Deepspeed on Intel Gaudi. We will use PyTorch for model training and Ray for distributed training. We will use dataset [tatsu-lab/alpaca](https://huggingface.co/datasets/tatsu-lab/alpaca).

[Intel Gaudi AI Processors (HPUs)](https://habana.ai) are AI hardware accelerators designed by Habana Labs. For more information, see [Gaudi Architecture](https://docs.habana.ai/en/latest/Gaudi_Overview/index.html) and [Gaudi Developer Docs](https://developer.habana.ai/).

Basic features for this fine-tuning example are:
- Running on HPUs, support three execution mode: ["lazy", "eager", "eager.compile"](https://docs.habana.ai/en/latest/PyTorch/Reference/PyTorch_Gaudi_Theory_of_Operations.html).
- Deepspeed integrated and LoRA training.
- [`GaudiTrainer`](https://github.com/huggingface/optimum-habana/blob/main/optimum/habana/transformers/trainer.py) based training.
- Llama-2-70b model.
- Ray based scheduling and management.

## Prepare environment
This example run on single node with 8 HPUs.

We recommend using a prebuilt container to run these examples. To run a container, you need Docker. See [Install Docker Engine](https://docs.docker.com/engine/install/) for installation instructions.

Next, follow [Run Using Containers](https://docs.habana.ai/en/latest/Installation_Guide/Bare_Metal_Fresh_OS.html?highlight=installer#run-using-containers) to install the Habana drivers and container runtime.

### Get docker image
``` bash
docker pull vault.habana.ai/gaudi-docker/1.15.1/ubuntu22.04/habanalabs/pytorch-installer-2.2.0:latest
```
### Run docker image
``` bash
docker run -it --runtime=habana -e HABANA_VISIBLE_DEVICES=all -e OMPI_MCA_btl_vader_single_copy_mechanism=none --cap-add=sys_nice --net=host --ipc=host vault.habana.ai/gaudi-docker/1.15.1/ubuntu22.04/habanalabs/pytorch-installer-2.2.0:latest
# maybe should mapping your workspace volumns
```
### Install dependency
``` bash
# for exection mode "eager" or "eager.compile", please install "optimum-habana>1.11.1"
pip install ray[train] notebook transformers datasets evaluate peft accelerate scikit-learn optimum-habana
# install deepspeed
pip install git+https://github.com/HabanaAI/DeepSpeed.git@1.15.1

# this notebook verfied with packages' version:
# transformers==4.38.2
# datasets==2.19.1
# evaluate==0.4.2
# peft==0.4.0
# accelerate==0.27.2
# scikit-learn==1.4.2
# optimum-habana==1.11.1
# deepspeed==0.12.4+hpu.synapse.v1.15.0
```

In [1]:
import os
import copy
import time
from typing import Dict

import torch
from torch import nn
from torch.utils.data import DataLoader

import datasets
import transformers
from transformers import (
    Trainer,
    TrainingArguments,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    AutoModelForSequenceClassification,
)

from tqdm import tqdm

import peft

from optimum.habana import GaudiTrainer, GaudiConfig, GaudiTrainingArguments
from optimum.habana.transformers.modeling_utils import adapt_transformers_to_gaudi

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Prepare Dataset Function

Preprocessing the raw dataset's each line with specified format.

In [2]:

def preprocess_dataset(raw_datasets):

    PROMPT_DICT = {
        "prompt_with_input": (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
        ),
        "prompt_without_input": (
            "Below is an instruction that describes a task. "
            "Write a response that appropriately completes the request.\n\n"
            "### Instruction:\n{instruction}\n\n### Response:"
        ),
    }

    def create_prompts(examples):
        prompts = {}
        prompts["source"] = []
        prompts["target"] = []
        for example in examples:
            prompt_template = (
                PROMPT_DICT["prompt_with_input"] if example["input"] != "" else PROMPT_DICT["prompt_without_input"]
            )
            source = prompt_template.format_map(example)
            prompts["source"].append(source)
            prompts["target"].append(example["output"])
        return prompts

    # Preprocessing the datasets.
    for key in raw_datasets:
        prompts = create_prompts(raw_datasets[key])
        columns_to_be_removed = list(raw_datasets[key].features.keys())
        raw_datasets[key] = raw_datasets[key].add_column("prompt_sources", prompts["source"])
        raw_datasets[key] = raw_datasets[key].add_column("prompt_targets", prompts["target"])
        raw_datasets[key] = raw_datasets[key].remove_columns(columns_to_be_removed)

## Dataset to Tokenizer Function

Tokenize each line in dataset by model tokenizer.

In example codes, we concatenate the dataset's line content to accelerate training speed.

All datasets are processed as "train" datasets, no evaluation datasets are sampled from raw_datasets.

In [3]:

def preprocess_dataset_to_tokenizer(raw_datasets, tokenizer):
    max_seq_length = 512
    tokenizer.pad_token_id = 0
    tokenizer.eos_token_id = 1
    tokenizer.bos_token_id = 2

    def tokenize(prompt, add_eos_token=True):
        results = tokenizer(
            prompt,
            truncation=True,
            max_length=max_seq_length,
            padding=False,
            return_tensors=None,
        )
        for i in range(len(results["input_ids"])):
            if (
                results["input_ids"][i][-1] != tokenizer.eos_token_id
                and len(results["input_ids"][i]) < max_seq_length
                and add_eos_token
            ):
                results["input_ids"][i].append(tokenizer.eos_token_id)
                results["attention_mask"][i].append(1)

        results["labels"] = copy.deepcopy(results["input_ids"])
        results["input_id_len"] = [len(result) for result in results["input_ids"]]
        return results

    def preprocess_function(examples):
        keys = list(examples.data.keys())
        if len(keys) != 2:
            raise ValueError("Unsupported dataset format")

        st = [s + t for s, t in zip(examples[keys[0]], examples[keys[1]])]

        examples_tokenized = tokenize(st)
        input_ids = examples_tokenized["input_ids"]
        labels = examples_tokenized["labels"]
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": examples_tokenized["attention_mask"],
        }

    tokenized_datasets = raw_datasets.map(
        preprocess_function,
        batched=True,
        load_from_cache_file=True,
    )

    def concatenate_data(dataset, max_seq_length):
        concatenated_dataset = {}
        for column in dataset.features:
            concatenated_data = [item for sample in dataset[column] for item in sample]
            reshaped_data = [
                concatenated_data[i * max_seq_length : (i + 1) * max_seq_length]
                for i in range(len(concatenated_data) // max_seq_length)
            ]
            concatenated_dataset[column] = reshaped_data
        return datasets.Dataset.from_dict(concatenated_dataset)

    tokenized_datasets_ = tokenized_datasets["train"].remove_columns(["prompt_sources", "prompt_targets"])
    tokenized_datasets["train"] = concatenate_data(tokenized_datasets_, max_seq_length)

    return tokenized_datasets

## Prepare training arguments

By instance object of `GaudiTrainingArguments`, the essential of initialization HPU will be called, such as HPU device spcification.

In [4]:

def prepare_training_args(config: Dict):
    # prepare execution mode config
    execution_mode = config["execution_mode"]
    use_lazy_mode = True if execution_mode == "lazy" else False
    torch_compile_backend = "hpu_backend" if execution_mode == "eager.compile" else None

    return GaudiTrainingArguments(deepspeed=config["deepspeed"],
                                  output_dir=config["output"],
                                  do_train=True,
                                  do_eval=False,
                                  per_device_train_batch_size=config["batch_size_per_worker"],
                                  bf16=True,
                                  learning_rate=config["lr"],
                                  save_strategy="no",
                                  torch_compile_backend=torch_compile_backend,
                                  evaluation_strategy="no",
                                  lr_scheduler_type="cosine",
                                  num_train_epochs=config["epochs"],
                                  use_lazy_mode=use_lazy_mode,
                                  use_habana=True,
                                  pipelining_fwd_bwd=True,
                                  save_only_model=True,
                                  gradient_checkpointing=True,
                                  warmup_ratio=0.03,
                                  throughput_warmup_steps=3,
                                  logging_steps=5)

## Training Function

This function will be executed by each worker during training, with following steps:

- loading datasets and preprocess datasets, just load the first 4096 item as training datasets.
- loading pretrained model as tokenizer, and process datasets to tokenizer.
- loading pretrained model, convert to lora model, and move model to HPU device.
- preparing data collator.
- preparing training args, an instance of `GaudiTrainingArguments`.
- preparing instance of `GaudiTrainer`.
- calling `train()` to train model.
- saving model results.


Compared to a training function for GPU, no changes are needed to port to HPU. Internally, Ray Train does these things:

- Detect HPU and set the device.
- Initialize the habana PyTorch backend.
- Initialize the habana distributed backend.

In [5]:

def train_func_per_worker(config: Dict):
    # adapt transformers to gaudi
    adapt_transformers_to_gaudi()

    # prepare training arguments
    training_args = prepare_training_args(config)

    # prepare datasets
    # here we use dataset "tatsu-lab/alpaca" from huggingface
    # and sample some part
    raw_datasets = datasets.DatasetDict({"train": datasets.load_dataset("tatsu-lab/alpaca", split='train[0:4096]')})
    preprocess_dataset(raw_datasets)

    # prepare tokenizer
    tokenizer = AutoTokenizer.from_pretrained(config["model"])
    tokenized_datasets = preprocess_dataset_to_tokenizer(raw_datasets, tokenizer)

    # prepare model
    if config["deepspeed"] is not None:
        auto_config = AutoConfig.from_pretrained(config["model"], use_cache=False, revision="main", use_auth_token=None, trust_remote_code=None)
        model = AutoModelForCausalLM.from_pretrained(config["model"], config=auto_config, **config["model_config"])
        model.generation_config.attn_softmax_bf16 = True
        model.generation_config.use_flash_attention = True
    else:
        model = AutoModelForCausalLM.from_pretrained(config["model"], **config["model_config"])

    peft_config = peft.LoraConfig(**config["lora_config"])
    model.enable_input_require_grads()
    model = peft.get_peft_model(model, peft_config)
    device = training_args.device
    model.to(dtype=config["model_config"]["torch_dtype"], device=device)

    # prepare data collator
    data_collator = DataCollatorForLanguageModeling(tokenizer, pad_to_multiple_of=8, return_tensors="pt", mlm=False)

    gaudi_config = GaudiConfig()
    gaudi_config.use_fused_adam = True
    gaudi_config.use_fused_clip_norm = True

    trainer = GaudiTrainer(
        model=model,
        gaudi_config=gaudi_config,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=None,
        preprocess_logits_for_metrics=None,
    )

    train_result = trainer.train()
    print(f"train_result = {train_result}")
    trainer.save_model()

    return train_result

## Main Training Function
The `train_llama` function sets up the distributed training environment using Ray and starts the training process. To enable training using HPU, we only need to make the following changes:
- Set the exectuion mode for training, supported execution mode are:

    - "lazy": Deferred execution of graphs, comprising of ops delivered from script op by op similar to Eager mode. It gives the Eager mode experience with performance on Gaudi. Unlike Eager Mode with torch.compile, graph is analyzed in each iteration leading to a higher CPU usage.
    - "eager": Op-by-op execution as defined in standard PyTorch Eager mode scripts.
    - "eager.compile": Eager mode extended with `torch.compile` - Similar to Eager mode but extended with wrapping complete or part of model (such as a function) into a graph. Parts that are not wrapped are executed eagerly.

    More detail theory can be found [here](https://docs.habana.ai/en/latest/PyTorch/Reference/PyTorch_Gaudi_Theory_of_Operations.html), and detail performance results can be found [here](https://developer.habana.ai/get-started/habana-models-performance/)
- Require an HPU for each worker in ScalingConfig
- Set backend to `hccl` in TorchConfig

In [6]:

def train_llama(num_workers, execution_mode):
    import ray
    from ray.train import ScalingConfig
    from ray.train.torch import TorchTrainer, TorchConfig

    # deepspeed config, can also place it to config file
    deepspeed_config = {
        "steps_per_print": 64,
        "train_batch_size": "auto",
        "train_micro_batch_size_per_gpu": "auto",
        "gradient_accumulation_steps": "auto",
        "bf16": {
            "enabled": True
        },
        "gradient_clipping": 1.0,
        "zero_optimization": {
            "stage": 3,
            "overlap_comm": False,
            "contiguous_gradients": False,
            "stage3_gather_16bit_weights_on_model_save": True
        }
    }

    # Preparing train configurations
    train_config = {
        "execution_mode": execution_mode,
        "model": "/root/models/models--meta-llama--Llama-2-70b-chat-hf/snapshots/e9149a12809580e8602995856f8098ce973d1080/",
        "model_config": {"torch_dtype": torch.bfloat16, "trust_remote_code": None, "use_auth_token": None},
        "lora_config": {"task_type": "CAUSAL_LM", "r": 4, "lora_dropout": 0.1, "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"]},
        "lr": 0.0018,
        "epochs": 2,
        "batch_size_per_worker": 10,
        "output": "/tmp/ray/",
        "deepspeed": deepspeed_config,
    }

    # Configure computation resources
    # In ScalingConfig, require an HPU for each worker
    scaling_config = ScalingConfig(num_workers=num_workers,
                                   use_gpu=False,
                                   resources_per_worker={"CPU": 1, "HPU": 1})
    # Set backend to hccl in TorchConfig
    torch_config = TorchConfig(backend="hccl")

    # start your ray cluster
    ray.init()

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        torch_config=torch_config,
        scaling_config=scaling_config,
    )

    result = trainer.fit()
    print(f"Training result: {result}")

## Start Training

Finally, we call the `train_llama` function to start the training process. You can adjust the number of workers to use, and the execution mode for HPU.

In [7]:
# set some environment variables
os.environ["RAY_EXPERIMENTAL_NOSET_HABANA_VISIBLE_MODULES"] = "0"
os.environ["PT_HPU_MAX_COMPOUND_OP_SIZE"] = "10"
os.environ["DEEPSPEED_HPU_ZERO3_SYNC_MARK_STEP_REQUIRED"] = "1"
# execution_mode are ["lazy", "eager", "eager.compile"]
execution_mode = "lazy"
os.environ["PT_HPU_LAZY_MODE"] = "1" if execution_mode == "lazy" else "0"
train_llama(num_workers=8, execution_mode="lazy")

2024-05-08 01:35:23,594	INFO worker.py:1749 -- Started a local Ray instance.
2024-05-08 01:35:24,779	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2024-05-08 01:35:24 (running for 00:00:00.11)
Using FIFO scheduling algorithm.
Logical resource usage: 0/152 CPUs, 0/0 GPUs (0.0/1.0 TPU, 0.0/8.0 HPU)
Result logdir: /tmp/ray/session_2024-05-08_01-35-20_406541_689603/artifacts/2024-05-08_01-35-24/TorchTrainer_2024-05-08_01-35-24/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(pid=699623) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(pid=699623)   _torch_pytree._register_pytree_node(
(TrainTrainable pid=699623) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(TrainTrainable pid=699623)   _torch_pytree._register_pytree_node(
(TrainTrainable pid=699623) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(TrainTrainable pid=699623)   _torch_pytree._register_pytree_node(


== Status ==
Current time: 2024-05-08 01:35:29 (running for 00:00:05.15)
Using FIFO scheduling algorithm.
Logical resource usage: 9.0/152 CPUs, 0/0 GPUs (0.0/1.0 TPU, 8.0/8.0 HPU)
Result logdir: /tmp/ray/session_2024-05-08_01-35-20_406541_689603/artifacts/2024-05-08_01-35-24/TorchTrainer_2024-05-08_01-35-24/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(RayTrainWorker pid=700358) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(RayTrainWorker pid=700358)   _torch_pytree._register_pytree_node(
(RayTrainWorker pid=700363) /usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
(RayTrainWorker pid=700363)   _torch_pytree._register_pytree_node(
(RayTrainWorker pid=700357) Setting up process group for: env:// [rank=0, world_size=8]


...

(RayTrainWorker pid=700357) {'loss': 0.8973, 'grad_norm': 0.07026992738246918, 'learning_rate': 7.096768816970011e-06, 'epoch': 1.92, 'memory_allocated (GB)': 16.85, 'max_memory_allocated (GB)': 29.6, 'total_memory_available (GB)': 94.62}
== Status ==
Current time: 2024-05-08 01:46:12 (running for 00:10:47.38)
Using FIFO scheduling algorithm.
Logical resource usage: 9.0/152 CPUs, 0/0 GPUs (8.0/8.0 HPU, 0.0/1.0 TPU)
Result logdir: /tmp/ray/session_2024-05-08_01-35-20_406541_689603/artifacts/2024-05-08_01-35-24/TorchTrainer_2024-05-08_01-35-24/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2024-05-08 01:46:17 (running for 00:10:52.40)
Using FIFO scheduling algorithm.
Logical resource usage: 9.0/152 CPUs, 0/0 GPUs (8.0/8.0 HPU, 0.0/1.0 TPU)
Result logdir: /tmp/ray/session_2024-05-08_01-35-20_406541_689603/artifacts/2024-05-08_01-35-24/TorchTrainer_2024-05-08_01-35-24/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




100%|██████████| 26/26 [07:35<00:00, 17.53s/it]


(RayTrainWorker pid=700363) train_result = TrainOutput(global_step=26, training_loss=1.0912420657964854, metrics={'train_runtime': 455.8151, 'train_samples_per_second': 8.352, 'train_steps_per_second': 0.11, 'total_flos': 113117358981120.0, 'train_loss': 1.0912420657964854, 'epoch': 2.0, 'memory_allocated (GB)': 16.85, 'max_memory_allocated (GB)': 29.51, 'total_memory_available (GB)': 94.62})
(RayTrainWorker pid=700357) {'train_runtime': 455.6666, 'train_samples_per_second': 8.352, 'train_steps_per_second': 0.11, 'train_loss': 1.0917885830769172, 'epoch': 2.0, 'memory_allocated (GB)': 16.85, 'max_memory_allocated (GB)': 29.6, 'total_memory_available (GB)': 94.62}
(RayTrainWorker pid=700357) train_result = TrainOutput(global_step=26, training_loss=1.0917885830769172, metrics={'train_runtime': 455.6666, 'train_samples_per_second': 8.352, 'train_steps_per_second': 0.11, 'train_loss': 1.0917885830769172, 'epoch': 2.0, 'memory_allocated (GB)': 16.85, 'max_memory_allocated (GB)': 29.6, 'tota

(RayTrainWorker pid=700357) /usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /root/models/models--meta-llama--Llama-2-70b-chat-hf/snapshots/e9149a12809580e8602995856f8098ce973d1080/ - will assume that the vocabulary was not modified.
(RayTrainWorker pid=700357)   warnings.warn(
2024-05-08 01:47:39,967	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-05-08 01:47:39,968	INFO tune.py:1007 -- Wrote the latest ve

Trial TorchTrainer_3de91_00000 completed. Last result: 
== Status ==
Current time: 2024-05-08 01:47:39 (running for 00:12:15.18)
Using FIFO scheduling algorithm.
Logical resource usage: 9.0/152 CPUs, 0/0 GPUs (8.0/8.0 HPU, 0.0/1.0 TPU)
Result logdir: /tmp/ray/session_2024-05-08_01-35-20_406541_689603/artifacts/2024-05-08_01-35-24/TorchTrainer_2024-05-08_01-35-24/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)


Training result: Result(
  metrics={},
  path='/root/ray_results/TorchTrainer_2024-05-08_01-35-24/TorchTrainer_3de91_00000_0_2024-05-08_01-35-24',
  filesystem='local',
  checkpoint=None
)
